In [ ]:
!pwd

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import scipy.ndimage as ndimage

In [ ]:
import os
import sys
sys.path.append('/sdf/home/l/lgupta/lgupta/github/lcls-tools/')
sys.path.append('/sdf/home/l/lgupta/lgupta/github/lcls-tools/lcls_tools/devices/profile_monitor')
sys.path.append('/sdf/home/l/lgupta/lgupta/github/lcls-tools/lcls_tools/devices/magnet')
sys.path.append('/sdf/home/l/lgupta/lgupta/github/lcls-tools/lcls_tools/image_processing')

from lcls_tools import image_processing as imp
import ImageProcessing as mip

In [ ]:

import glob

files = []

path = "/gpfs/slac/staas/fs1/g/accelerator_modeling/lgupta/github/lcls_sc_inj_surrogate/measurements/files/"

for file in glob.glob(path+"ProfMon-C*.mat"):
    files.append(os.path.join(path, file))

    
print(len(files))

In [ ]:
files

In [ ]:
image = imp.mat_image.MatImage()
image.load_mat_image(files[0])
images = image.image

for f in files[1:]:
    image = imp.mat_image.MatImage()
    image.load_mat_image(f)
    images = np.dstack((images, image.image))
    resolution = image.resolution

print("Resolution is = " + str(resolution))
print(np.shape(images))    

In [ ]:
VCC_dict = mip.doAnalysisAndReturnDict(images, plotting = True, VCC = True, npix = 150, resolution = resolution, verbose = False, scale_factor = 5)

In [ ]:
savepath

In [ ]:
savepath = path
savenames = []

for ind, name in enumerate(files):
    name = name.split('.')[0]
    savename = savepath+"file_"+str(ind)
    image = VCC_dict['masked_images'][ind]
    n,m = np.shape(image)
    np.savetxt(savename+'.txt',image)
    xwidth=resolution*n*(1E-3)
    ywidth=resolution*m*(1E-3)
    xcm=resolution*(n/2)*(1E-3)
    ycm=resolution*(m/2)*(1E-3)
    #print( xwidth, ywidth, xcm, ycm)
    with open(savename+'.txt', 'r+') as f:
        with open(savename+"_out.txt", "w") as f2:
            line = 'x ' + str(xwidth) + " " + str(xcm) + " [mm]\ny " + str(ywidth) + " " + str(ycm) + " [mm]\n"
            f2.write(line)
            for line in f:
                f2.write(line)
    
    savenames.append(savename+"_out.txt")
    
    

In [ ]:
# import yaml
# ####for making yaml input files DO NOT DELETE
# originalfile = "/global/homes/l/lgupta/m669/MeasuredData/VCC/laser_in.yaml"
# with open(originalfile) as j:
#     originaldata = yaml.load(j, Loader=yaml.FullLoader)
    
# yaml_input_files = []
# for ind, name in enumerate(savenames):
#     data = originaldata
#     data["xy_dist"]["file"] = name
#     data["start"]["MTE"]["value"] = 250
#     outname = name.split('_out')[0] + "_laser_in.yaml"
#     yaml_input_files.append(outname)
#     with open(outname, 'w') as f:
#         yaml.dump(data, f)

# print(len(yaml_input_files))
# print("Files written to:  "+savepath)
